<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/bigtweet_ask_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

In [ ]:
base_dir = 'medical-concept-normalization'

In [ ]:
os.mkdir(base_dir)

In [ ]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [ ]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [ ]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [ ]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [ ]:
path = Path('medical-concept-normalization/data_collection')

In [ ]:
# training data set

train_csv = path/'AskAPatient.fold-2.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [ ]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-2.validation.csv",header=None)
valid.head()

,0,1
0,2,left heel pain
1,4,Discoloration of skin [blotchy] on the neck
2,7,goiter
3,8,arthritis
4,8,arthritis


In [ ]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-2.test.csv",header=None)
test.head()

,0,1
0,2,right heel pain
1,4,black and blue marks
2,8,arthritice
3,8,arthritis
4,8,arthritis


In [ ]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'bigtweet_ask.csv')

In [ ]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [ ]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [ ]:
data_clas.add_test(test,label=0)


In [ ]:
data_lm.save()
data_clas.save()

In [ ]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'pain',
 'lipitor',
 'in',
 'of',
 'muscle',
 'to',
 'the',
 'severe',
 'and',
 'my',
 'myalgia',
 'fatigue',
 'lower',
 'rt',
 'a',
 'cramp',
 'loss',
 'limb',
 'weakness',
 'for',
 'stomach',
 'disease',
 'back',
 '-',
 'depression',
 'on',
 'cramps',
 'gas',
 'upper',
 'is',
 'memory',
 'joint',
 'you',
 'leg',
 'legs',
 'pains',
 'i',
 'headache',
 'unable',
 'with',
 'shoulder',
 'arthralgia',
 'knee',
 'neck',
 'excessive',
 'foot',
 'aches',
 'gastrointestinal',
 'hip',
 'amp',
 'numbness',
 'not',
 'arthritis',
 'insomnia',
 'like',
 'all',
 '/',
 'bleeding',
 'arthrotec',
 'gluten',
 'walk',
 'asthenia',
 'dizziness',
 'tired',
 'nausea',
 'feeling',
 'abdominal',
 'problems',
 'walking',
 'heart',
 'feet',
 'difficulty',
 'muscles',
 'swelling',
 'chest',
 'ache',
 'hair',
 'backache',
 'at',
 'hand',
 'extreme',
 'burning',
 'zocor',
 'could',
 'stiffness',
 'diarrhoea',
 'can',
 'visi

In [ ]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [ ]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [ ]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.254325,2.766317,0.547308,19:11


In [ ]:
# learn.fit_one_cycle(1, 1e-2)

In [ ]:
# learn.fit_one_cycle(1, 1e-3)

In [ ]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,2.825487,2.708482,0.553125,19:14


In [ ]:
learn.save_encoder('bigtweet_ask_ask_fold2_first')

In [ ]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('bigtweet_ask_ask_fold2_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.041721,3.527359,0.352243,01:09


In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.099262,2.998225,0.488127,01:22


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.395055,2.638406,0.573879,03:09


In [ ]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.103515,2.334264,0.606860,03:07
1,1.753989,2.186458,0.647757,03:01


In [ ]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.582257,1.974054,0.674142,03:01
1,1.400741,1.884095,0.700528,03:10


In [ ]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.287917,1.780498,0.697889,03:12
1,1.166370,1.699860,0.720317,03:06


In [ ]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.075864,1.614104,0.736148,03:00
1,0.929274,1.544817,0.740106,03:15


In [ ]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.905682,1.501490,0.753298,03:24
1,0.741007,1.500712,0.758575,03:18


In [ ]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.696199,1.416731,0.765172,03:05
1,0.694595,1.373504,0.765172,03:06
2,0.624625,1.299672,0.774406,03:12
3,0.503104,1.294074,0.782322,03:21


In [ ]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.515337,1.299722,0.781003,03:14
1,0.531494,1.250641,0.792876,03:02
2,0.465688,1.262557,0.790237,03:11
3,0.376458,1.266784,0.798153,03:23


In [ ]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.412636,1.252278,0.792876,03:15
1,0.433458,1.284217,0.802111,03:13
2,0.363783,1.224852,0.803430,03:16
3,0.320127,1.209009,0.808707,03:24


In [ ]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.358402,1.241647,0.798153,03:18
1,0.360825,1.283335,0.795515,03:21
2,0.314008,1.287012,0.807388,03:07
3,0.283917,1.245282,0.806069,03:01


In [ ]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.261775,1.298254,0.806069,03:11
1,0.340206,1.289389,0.811346,03:06
2,0.272509,1.288798,0.810026,03:05
3,0.227109,1.280907,0.813984,03:08


In [ ]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.250820,1.245810,0.812665,03:26
1,0.274218,1.332309,0.821900,03:12
2,0.261009,1.258817,0.816623,03:17
3,0.203112,1.283311,0.820580,03:15


In [ ]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.229779,1.329546,0.817942,03:20
1,0.254460,1.303493,0.817942,03:11
2,0.222182,1.309173,0.816623,03:22
3,0.199777,1.266375,0.816623,03:02


In [ ]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.212572,1.433434,0.821900,03:23
1,0.200883,1.387193,0.823219,03:31


In [ ]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.206745,1.358443,0.819261,03:27
1,0.196074,1.360647,0.820580,03:01


In [ ]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.182465,1.417626,0.824538,03:26


In [ ]:
# save the best model

learn.save_encoder('bigtweet_ask_ask_mixed_fold2')

# Part three: Predict on the test dataset

In [ ]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [ ]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[2, 821, 1000, 8, 8, 8, 179, 8, 554, 16, 836, 27, 30, 86, 414, 414, 414, 32, 34, 37, 37, 741, 37, 37, 37, 37, 37, 37, 37, 37, 39, 623, 44, 922, 49, 94, 49, 325, 49, 49, 49, 49, 49, 49, 49, 891, 52, 53, 370, 370, 53, 309, 58, 60, 62, 62, 179, 752, 70, 70, 70, 71, 71, 71, 996, 76, 289, 79, 79, 614, 546, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 861, 861, 1022, 79, 79, 79, 79, 220, 80, 80, 80, 80, 80, 80, 531, 83, 268, 325, 92, 92, 850, 94, 94, 94, 826, 826, 94, 94, 94, 94, 94, 96, 96, 594, 246, 100, 100, 100, 100, 108, 108, 108, 289, 275, 111, 111, 111, 111, 211, 111, 214, 519, 125, 261, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 172, 131, 132, 135, 316, 316, 89, 139, 142, 143, 738, 151, 151, 152, 736, 157, 161, 162, 163, 166, 166, 172, 172, 172, 172, 172, 172, 205, 54, 180, 182, 182, 182, 182, 183, 183, 100, 187, 187, 187, 187, 187, 189, 190, 190, 190, 190, 190, 309, 190, 190, 193, 193, 170, 197, 197, 1

In [ ]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

662
0.7644341801385681
